In [13]:
print "IMPORTING LIBRARIES..."
import pandas as pd
import numpy as np
import statsmodels.api as sm
import re
import requests
from requests.auth import HTTPBasicAuth


#DOWLOADING FILE FROM DROPBOX FIRST TIME
import urllib2
import os.path
import time
import random
while not os.path.exists('dev.csv') or not os.path.exists('oot0.csv'):
    time.sleep (3*random.random()); #Sleeping less than 3 seconds before going to Dropbox - avoid too many students at once.
    if not os.path.exists('dev.csv'):
        print "DOWLOADING FILE dev.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/dev.csv")
        output = open('dev.csv','wb')
        output.write(csvfile.read())
        output.close()
    if not os.path.exists('oot0.csv'):
        print "DOWLOADING FILE oot0.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/oot0.csv")
        output = open('oot0.csv','wb')
        output.write(csvfile.read())
        output.close()  
#DOWLOADING FILE FROM DROPBOX FIRST TIME

    
print "LOADING DATASETS..."
df = pd.read_csv("dev.csv") #DEV-SAMPLE
dfo = pd.read_csv("oot0.csv")#OUT-OF-TIME SAMPLE

print "IDENTIFYING TYPES..."
in_model = []
list_ib = set()  #input binary
list_icn = set() #input categorical nominal
list_ico = set() #input categorical ordinal
list_if = set()  #input numerical continuos (input float)
list_inputs = set()
output_var = 'ob_target'

for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
        print var_name,"is input"   
    if re.search('^ib_',var_name):
        list_ib.add(var_name)
        print var_name,"is input binary"
    elif re.search('^icn_',var_name):
        list_icn.add(var_name)
        print var_name,"is input categorical nominal"
    elif re.search('^ico_',var_name):
        list_ico.add(var_name)
        print var_name,"is input categorical ordinal"
    elif re.search('^if_',var_name):
        list_if.add(var_name)
        print var_name,"is input numerical continuos (input float)"
    elif re.search('^ob_',var_name):
        output_var = var_name
    else:
        print "ERROR: unable to identify the type of:", var_name


print "STEP 1: DOING MY TRANSFORMATIONS..."
#Será necesario transformar las variables categoricas ordinales, en variables 

print "STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL..."
#in_model = list_inputs #['ib_var_1','icn_var_22','ico_var_25','if_var_65']
#in_model = ['ib_var_1','icn_var_22','ico_var_25','if_var_65']
####-------------------------------------------#####
from sklearn.ensemble import RandomForestClassifier

list_inputs = list(set(list_inputs))
X = df[list_inputs]
y = df[output_var]

print "STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST..."
# Build a forest and compute the feature importances
forest = RandomForestClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
in_model = set()
print("Feature ranking:")
for f in range(70):
    print("%d. feature %s (%f) " % (f + 1, list_inputs[indices[f]], importances[indices[f]] ))
    in_model.add(list_inputs[indices[f]])

in_model = list(in_model) 
print in_model
####-------------------------------------------#####

print "STEP 3: DEVELOPING THE MODEL..."
X = df[list(in_model)]
y = df[output_var]

Xo = dfo[list(set(in_model))]

#model = sm.Logit(y,X)
try:
    model = sm.GLM(y,X, family = sm.families.Binomial())
    result = model.fit()
    print result.summary()
    y_pred = result.predict(X)
    yo_pred = result.predict(Xo)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

print "STEP 4: ASSESSING THE MODEL..."
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(df[output_var], y_pred)-1
print "GINI DEVELOPMENT=", gini_score

print "STEP 5: SUBMITTING THE RESULTS..."
dfo['pred'] = yo_pred
dfo_tosend = dfo[list(['id','pred'])]

i=1
filename = "group_Z_sub"+str(i)+".csv"
dfo_tosend.to_csv(filename, sep=',')

url = 'http://mgadi.pythonanywhere.com/api/v1.0/uploadpredictions'

files = {'file': (filename, open(filename, 'rb'))}
rsub = requests.post(url, files=files, auth=HTTPBasicAuth('amarino', 'amarino'))
resp_str = str(rsub.text)
print "RESULT SUBMISSION: ", resp_str





IMPORTING LIBRARIES...
LOADING DATASETS...
IDENTIFYING TYPES...
id is input
ERROR: unable to identify the type of: id
ib_var_1 is input
ib_var_1 is input binary
ib_var_2 is input
ib_var_2 is input binary
ib_var_3 is input
ib_var_3 is input binary
ib_var_4 is input
ib_var_4 is input binary
ib_var_5 is input
ib_var_5 is input binary
ib_var_6 is input
ib_var_6 is input binary
ib_var_7 is input
ib_var_7 is input binary
ib_var_8 is input
ib_var_8 is input binary
ib_var_9 is input
ib_var_9 is input binary
ib_var_10 is input
ib_var_10 is input binary
ib_var_11 is input
ib_var_11 is input binary
ib_var_12 is input
ib_var_12 is input binary
ib_var_13 is input
ib_var_13 is input binary
ib_var_14 is input
ib_var_14 is input binary
ib_var_15 is input
ib_var_15 is input binary
ib_var_16 is input
ib_var_16 is input binary
ib_var_17 is input
ib_var_17 is input binary
ib_var_18 is input
ib_var_18 is input binary
ib_var_19 is input
ib_var_19 is input binary
ib_var_20 is input
ib_var_20 is input binary


In [14]:
print "STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL..."
#in_model = list_inputs #['ib_var_1','icn_var_22','ico_var_25','if_var_65']
#in_model = ['ib_var_1','icn_var_22','ico_var_25','if_var_65']
####-------------------------------------------#####
from sklearn.ensemble import ExtraTreesClassifier

list_inputs = list(set(list_inputs))
X = df[list_inputs]
y = df[output_var]

print "STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST..."
# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
in_model = set()
print("Feature ranking:")
for f in range(70):
    print("%d. feature %s (%f) " % (f + 1, list_inputs[indices[f]], importances[indices[f]] ))
    in_model.add(list_inputs[indices[f]])

in_model = list(in_model) 
print in_model
####-------------------------------------------#####

print "STEP 3: DEVELOPING THE MODEL..."
X = df[list(in_model)]
y = df[output_var]

Xo = dfo[list(set(in_model))]

#model = sm.Logit(y,X)
try:
    model = sm.GLM(y,X, family = sm.families.Binomial())
    result = model.fit()
    print result.summary()
    y_pred = result.predict(X)
    yo_pred = result.predict(Xo)
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

print "STEP 4: ASSESSING THE MODEL..."
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(df[output_var], y_pred)-1
print "GINI DEVELOPMENT=", gini_score

STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL...
STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST...
Feature ranking:
1. feature if_var_68 (0.027557) 
2. feature if_var_79 (0.023182) 
3. feature icn_var_24 (0.020903) 
4. feature if_var_65 (0.019567) 
5. feature if_var_76 (0.019216) 
6. feature if_var_69 (0.019146) 
7. feature if_var_72 (0.019030) 
8. feature if_var_75 (0.018068) 
9. feature ico_var_35 (0.018049) 
10. feature if_var_71 (0.017968) 
11. feature if_var_67 (0.017520) 
12. feature ico_var_36 (0.017491) 
13. feature ico_var_34 (0.017350) 
14. feature if_var_81 (0.016739) 
15. feature id (0.016420) 
16. feature if_var_70 (0.016268) 
17. feature if_var_78 (0.016028) 
18. feature ico_var_39 (0.015556) 
19. feature ico_var_31 (0.015521) 
20. feature ico_var_29 (0.015490) 
21. feature ico_var_56 (0.015261) 
22. feature if_var_77 (0.014877) 
23. feature if_var_80 (0.014814) 
24. feature ico_var_47 (0.014796) 
25. feature ico_var_52 (0.014651) 
26. feature icn_var_22 (0.0